In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# We need to join the upper directory in order to access the local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [30]:
import itertools
import json

In [31]:
import numpy as np
import pandas as pd
import scipy
from scipy import optimize

In [32]:
# Our precious pytdoa functions
import hacktdoa.geodesy.geodesy as geodesy
from hacktdoa.optim import nlls, exact, lls

from hacktdoa.util.util import generate_heatmap, generate_hyperbola

In [6]:
# Sensor and transmitter positions
receivers = pd.read_pickle('../data/positions/receivers.pkl')
transmitters = pd.read_pickle('../data/positions/transmitters.pkl')

In [33]:
receivers_enu = np.empty((0,3))

for enu_tuple in receivers.enu:
    receivers_enu = np.vstack((receivers_enu,np.asarray(enu_tuple)))

In [34]:
transmitters_enu = np.empty((0,3))

for enu_tuple in transmitters.enu:
    transmitters_enu = np.vstack((transmitters_enu,np.asarray(enu_tuple)))

In [35]:
# Get the data
data = pd.read_pickle('../data/sims/sim_pairs_250ns.pkl')

### Single Analysis

In [10]:
transmitter = 0 # 0, 1 & 2
tx_position = transmitters_enu[transmitter]
tx_data = data.loc[transmitter]

In [11]:
combinations = np.empty((0,2), dtype='int')
for tup in tx_data.columns:
    combinations = np.vstack((combinations,np.asarray(tup)))

In [12]:
receivers_enu

array([[-3.88190824e+02, -1.26919332e+01, -1.18048203e-02],
       [-5.16508660e+02, -4.41699504e+02, -3.61919706e-02],
       [ 1.55866745e+02, -5.41817320e+02, -2.49463465e-02],
       [ 6.12819967e+02,  3.59067674e+02, -3.95090348e-02],
       [ 1.35937006e+02,  6.37211434e+02, -3.33204327e-02]])

In [16]:
tdoas = tx_data.iloc[6].to_numpy()

In [13]:
dimensions = 2

sensors_mean = np.mean(receivers_enu, axis=0)[0:dimensions]

sensors = (receivers_enu[:,0:dimensions] - sensors_mean)
optimfun = lambda X: nlls.nlls_enu(X, sensors, -tdoas, combinations)

In [14]:
distances = np.sqrt(np.sum(np.square(sensors - tx_position[0:dimensions]), axis=1))

theoretical_tdoa = np.empty((0,))
for combo in combinations:
    theoretical_tdoa = np.append(theoretical_tdoa, distances[combo[0]]-distances[combo[1]])

In [17]:
print(-tdoas)
print(theoretical_tdoa)

[-6.24510413e+00  6.33695006e+02  4.87124656e+00 -2.66372193e+02
  7.79633832e+02 -2.03342205e-02 -2.77632147e+02 -5.78316704e+02
 -8.69310770e+02 -2.48501074e+02]
[ -25.53036486  611.54116988   27.82830276 -231.0560987   637.07153473
   53.35866761 -205.52573384 -583.71286712 -842.59726858 -258.88440146]


In [18]:
# Minimization routine
# If no initial point is given we start at the center
X0 = np.zeros(shape=(dimensions,))

summary = optimize.minimize(optimfun, X0, method="Nelder-Mead", options={"maxiter": 300})

if dimensions == 2 or dimensions == 3:
    res = np.array(summary.x, copy=False).reshape(-1, dimensions)
else:
    raise RuntimeError("[ERROR]: Only valid dimensions are 2 or 3.")

res = (res + sensors_mean).squeeze()

In [19]:
print(res)
print(summary.message)

[ 361.31309071 -457.57537793]
Optimization terminated successfully.


In [20]:
# Linear solutions
numc = 4
combs = combinations[:,0:numc]
if combs.shape[0] == 2:
    res = exact.fang(sensors[:,0:numc+1], -tdoas[0:numc])
else:
    res = lls.lls(sensors[:,0:numc+1], -tdoas[0:numc])

print(res)

[ 345.5793169  -461.27343512]


In [21]:
tx_position[0:dimensions]

array([ 354.84230259, -417.52578   ])

In [23]:
nlls.nlls_enu(tx_position[0:2], sensors, tdoas, combinations)

8581849.469007539

### Batch Processing

In [36]:
dimensions = 2
method = 'nonlinear'
results = np.empty((0,4 + dimensions))
X0 = np.zeros(shape=(dimensions,))

error_dist = [10, 100, 250, 1000]
for error_ns in error_dist:
    data = pd.read_pickle(f'../data/sims/sim_{error_ns}ns.pkl')

    # Get the combinations from the columns
    combinations = np.empty((0,2), dtype='int')
    for tup in data.loc[0]:
        combinations = np.vstack((combinations,np.asarray(tup)))

    sensors_mean = np.mean(receivers_enu, axis=0)[0:dimensions]

    sensors = (receivers_enu[:,0:dimensions] - sensors_mean)

    # We start the interesting stuff
    for transmitter in range(3):
        tx_position = transmitters_enu[transmitter][0:dimensions]
        tx_data = data.loc[transmitter]

        for row, tdoa_series in tx_data.iterrows():
            tdoas = -tdoa_series.to_numpy()

            if method == "nonlinear": 
                optimfun = lambda X: nlls.nlls_enu(X, sensors, tdoas, combinations)

                # For dimension 3, need to constrain up direction
                constraints = []
                if dimensions == 3:
                    A = np.eye(3)
                    lb = np.array([-100000, -100000, 0.0])
                    ub = np.array([100000, 100000, np.inf])
                    
                    constraints.append(optimize.LinearConstraint(A[0], lb=lb[0], ub=ub[0]))     
                    constraints.append(optimize.LinearConstraint(A[1], lb=lb[1], ub=ub[1]))
                    constraints.append(optimize.LinearConstraint(A[2], lb=lb[2], ub=ub[2]))

                if dimensions == 2:
                    summary = optimize.minimize(optimfun, X0, method="Nelder-Mead", constraints=constraints)
                elif dimensions == 3:
                    summary = optimize.minimize(optimfun, X0, method="SLSQP", constraints=constraints)
                res = np.array(summary.x, copy=False).reshape(-1, dimensions)
                success = summary.success

            elif method == "linear":
                if combinations.shape[0] == 2:
                    res = exact.fang(sensors, tdoas)
                else:
                    res = lls.lls(sensors, tdoas)
                
                if res is None:
                    success = False
                else:
                    success = True
            else:
                raise RuntimeError(f"Supported methods are linear|nonlinear. You selected {method}")

            res = (res + sensors_mean).squeeze()
            err = np.sqrt(np.sum(np.square(res - tx_position)))

            if dimensions == 2:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], err, success])))
            elif dimensions == 3:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], res[2], err, success])))

if dimensions == 2:
    results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'mse', 'success'])
elif dimensions == 3:   
    results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'up', 'mse', 'success'])

KeyboardInterrupt: 

In [56]:
results_pd.astype({"ns_err":int, "tx": int})

,ns_err,tx,east,north,mse,success
0,10,0,356.567111,-420.438888,3.385434,1.0
1,10,0,336.792758,-391.882574,31.358573,1.0
2,10,0,353.693886,-420.496342,3.184823,1.0
3,10,0,352.774438,-417.665739,2.072596,1.0
4,10,0,368.888330,-433.197138,21.044770,1.0
...,...,...,...,...,...,...
1195,1000,2,138.105350,119.311374,293.844933,1.0
1196,1000,2,152.016766,-75.367084,486.656255,1.0
1197,1000,2,109.678719,12.708358,392.897739,1.0
1198,1000,2,132.710108,-20.291659,428.990871,1.0


In [ ]:
results_pd.to_pickle(f'../data/results/sim_results_{dimensions}d_{method}.pkl')

### All pairs

In [37]:
dimensions = 2
method = 'nonlinear'
results = np.empty((0,4 + dimensions))
X0 = np.zeros(shape=(dimensions,))

error_dist = [250]
for error_ns in error_dist:
    data = pd.read_pickle(f'../data/sims/sim_pairs_{error_ns}ns.pkl')

    # Get the combinations from the columns
    combinations = np.empty((0,2), dtype='int')
    for tup in data.loc[0]:
        combinations = np.vstack((combinations,np.asarray(tup)))

    sensors_mean = np.mean(receivers_enu, axis=0)[0:dimensions]

    sensors = (receivers_enu[:,0:dimensions] - sensors_mean)

    # We start the interesting stuff
    for transmitter in range(3):
        tx_position = transmitters_enu[transmitter][0:dimensions]
        tx_data = data.loc[transmitter]

        for row, tdoa_series in tx_data.iterrows():
            tdoas = -tdoa_series.to_numpy()

            if method == "nonlinear": 
                optimfun = lambda X: nlls.nlls_enu(X, sensors, tdoas, combinations)

                # For dimension 3, need to constrain up direction
                constraints = []
                if dimensions == 3:
                    A = np.eye(3)
                    lb = np.array([-100000, -100000, 0.0])
                    ub = np.array([100000, 100000, np.inf])
                    
                    constraints.append(optimize.LinearConstraint(A[0], lb=lb[0], ub=ub[0]))     
                    constraints.append(optimize.LinearConstraint(A[1], lb=lb[1], ub=ub[1]))
                    constraints.append(optimize.LinearConstraint(A[2], lb=lb[2], ub=ub[2]))

                if dimensions == 2:
                    summary = optimize.minimize(optimfun, X0, method="Nelder-Mead", constraints=constraints)
                elif dimensions == 3:
                    summary = optimize.minimize(optimfun, X0, method="SLSQP", constraints=constraints)
                res = np.array(summary.x, copy=False).reshape(-1, dimensions)
                success = summary.success

            elif method == "linear":
                if combinations.shape[0] == 2:
                    res = exact.fang(sensors, tdoas)
                else:
                    res = lls.lls(sensors, tdoas)
                
                if res is None:
                    success = False
                else:
                    success = True
            else:
                raise RuntimeError(f"Supported methods are linear|nonlinear. You selected {method}")

            res = (res + sensors_mean).squeeze()
            err = np.sqrt(np.sum(np.square(res - tx_position)))

            if dimensions == 2:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], err, success])))
            elif dimensions == 3:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], res[2], err, success])))

if dimensions == 2:
    results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'mse', 'success'])
elif dimensions == 3:   
    results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'up', 'mse', 'success'])

In [38]:
results_pd.astype({"ns_err":int, "tx": int})

,ns_err,tx,east,north,mse,success
0,250,0,189.978249,-425.924298,165.077834,1.0
1,250,0,354.329572,-443.459751,25.939039,1.0
2,250,0,374.383575,-479.688810,65.162134,1.0
3,250,0,118.701004,-422.446880,236.192570,1.0
4,250,0,349.636761,-358.540038,59.214993,1.0
...,...,...,...,...,...,...
295,250,2,45.687506,460.496298,59.686445,1.0
296,250,2,108.679377,474.670520,87.690420,1.0
297,250,2,-7.329168,386.033432,68.406119,1.0
298,250,2,71.868742,331.385945,72.090527,1.0


In [39]:
results_pd.to_pickle(f'../data/results/sim_results_{dimensions}d_{method}_pairs.pkl')

### All receiver pairs

In [40]:
import os

In [53]:
dimensions = 2
method = 'nonlinear'
X0 = np.zeros(shape=(dimensions,))

num_sensors = 4
path = f"/home/ygglc/desktop/electrosense/hackathons/2022/hackathon_alicante/hacktdoa/data/sims/combinations_{num_sensors}"
for file in os.scandir(path):
    data = pd.read_pickle(os.path.join(path,file))

    results = np.empty((0,4 + dimensions))

    # Get the combinations from the columns
    combinations = np.empty((0,2), dtype='int')
    for tup in data.loc[0]:
        combinations = np.vstack((combinations,np.asarray(tup)))

    sensors_mean = np.mean(receivers_enu, axis=0)[0:dimensions]

    sensors = (receivers_enu[:,0:dimensions] - sensors_mean)

    # We start the interesting stuff
    for transmitter in range(3):
        tx_position = transmitters_enu[transmitter][0:dimensions]
        tx_data = data.loc[transmitter]

        for row, tdoa_series in tx_data.iterrows():
            tdoas = -tdoa_series.to_numpy()

            if method == "nonlinear": 
                optimfun = lambda X: nlls.nlls_enu(X, sensors, tdoas, combinations)

                # For dimension 3, need to constrain up direction
                constraints = []
                if dimensions == 3:
                    A = np.eye(3)
                    lb = np.array([-100000, -100000, 0.0])
                    ub = np.array([100000, 100000, np.inf])
                    
                    constraints.append(optimize.LinearConstraint(A[0], lb=lb[0], ub=ub[0]))     
                    constraints.append(optimize.LinearConstraint(A[1], lb=lb[1], ub=ub[1]))
                    constraints.append(optimize.LinearConstraint(A[2], lb=lb[2], ub=ub[2]))

                if dimensions == 2:
                    summary = optimize.minimize(optimfun, X0, method="Nelder-Mead", constraints=constraints)
                elif dimensions == 3:
                    summary = optimize.minimize(optimfun, X0, method="SLSQP", constraints=constraints)
                res = np.array(summary.x, copy=False).reshape(-1, dimensions)
                success = summary.success

            elif method == "linear":
                if combinations.shape[0] == 2:
                    res = exact.fang(sensors, tdoas)
                else:
                    res = lls.lls(sensors, tdoas)
                
                if res is None:
                    success = False
                else:
                    success = True
            else:
                raise RuntimeError(f"Supported methods are linear|nonlinear. You selected {method}")

            res = (res + sensors_mean).squeeze()
            err = np.sqrt(np.sum(np.square(res - tx_position)))

            if dimensions == 2:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], err, success])))
            elif dimensions == 3:
                results = np.vstack((results,np.array([error_ns, transmitter, res[0], res[1], res[2], err, success])))

    if dimensions == 2:
        results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'mse', 'success'])
    elif dimensions == 3:   
        results_pd = pd.DataFrame(results, columns=['ns_err', 'tx', 'east', 'north', 'up', 'mse', 'success'])

    results_pd.astype({"ns_err":int, "tx": int, "success": bool})
    result_file = file.name.split(".")[0]
    results_pd.to_pickle(f'../data/results/combinations_{num_sensors}/{result_file}_result.pkl')

In [54]:
results_pd

,ns_err,tx,east,north,mse,success
0,250.0,0.0,514.797189,-460.918566,165.736235,1.0
1,250.0,0.0,775.297876,-985.689013,706.818469,1.0
2,250.0,0.0,686.945529,-537.023012,352.947789,1.0
3,250.0,0.0,342.796333,-385.596402,34.126098,1.0
4,250.0,0.0,318.598741,-421.177527,36.427064,1.0
...,...,...,...,...,...,...
295,250.0,2.0,183.201272,344.748575,136.807360,1.0
296,250.0,2.0,73.245310,332.218020,71.531084,1.0
297,250.0,2.0,-68.864102,505.033656,164.081374,1.0
298,250.0,2.0,2.028028,412.539737,57.982102,1.0
